In [1]:
import json
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.registry import register_env
from fury_sim_env import FurySimEnv
from gymnasium import make
from gymnasium.envs.registration import register
from gymnasium.wrappers import FrameStack
from fury import *

spell_id_lists = {
        '0' : "Melee",
        '23881' : "Bloodthirst",
        '1680' : "Whirlwind",
        '47475' : "Slam",
        '47450' : "Heroic Strike",
        '47471' : "Execute",
        '12867': "Deep Wounds",
        '12292' : "Death Wish",
        '1719' : "Recklessness",
        '64382' : "Shattering Throw",
        '54758' : "Engi Gloves",
        '2457' : "Battle Stance",
        '2458' : "Berserker Stance",
        '2687' : 'Bloodrage',
        '2825' : 'Bloodlust',
        '47465': 'Rend',
        '7384': 'Overpower',
        '44949': 'Whirlwind OH',
}

c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Fury Raid Sim Request JSON

In [2]:
f = open('data/fury-human-bis-p3.json')
settings = json.load(f)

# Set environment and settings

In [3]:
def reset():
    wowsims.new(json.dumps(settings).encode('utf-8'))


# Iterations are currently capped at 3000
iterations = settings['simOptions']['iterations']

duration = settings['encounter']['duration']
reset()

# Default sim agent
Default sim agent is the hardcoded agent from the sim

In [4]:
settings['simOptions']['interactive'] = False
damages = np.array([])

for i in range(iterations):
    reset()
    while not wowsims.step():
        pass
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)

print(f'Average DPS: {damages.mean() / duration}')

Average DPS: 11891.96688184432


In [5]:
settings['simOptions']['interactive'] = False

reset()
while not wowsims.step():
    pass
    
    
cast_metrics = wowsims.getSpellMetrics()

for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 1, 1, 2, 9, 77]
Death Wish: [2]
Deep Wounds: [133]
Whirlwind: [16]
Recklessness: [1]
Bloodthirst: [33]
Battle Stance: [9]
Berserker Stance: [9]
Bloodrage: [4]
Bloodlust: [1]
Whirlwind OH: [16]
Heroic Strike: [68]
Rend: [4]
Execute: [8]
Slam: [19]
Engi Gloves: [3]
Shattering Throw: [1]
Overpower: [5]


# PPO Agent

## Load Trained PPO Model

In [3]:
config = PPOConfig()\
            .rollouts(num_rollout_workers=1)


register(id="FurySimEnv", entry_point="fury_sim_env:FurySimEnv")
env = make("FurySimEnv")
# env_creator = lambda config: FrameStack(env, num_stack=5)
env_creator = lambda config: FurySimEnv(...)
register_env("FurySimEnv", env_creator=env_creator)
# env = FrameStack(env, num_stack=5)

algorithm = config.build(env="FurySimEnv")

algorithm.from_checkpoint("models//test\PPO\PPO_FurySimEnv_9a49c_00000_0_2023-05-18_15-04-24\checkpoint_000012")

<>:14: DeprecationWarning: invalid escape sequence \P
c:\ProgramData\Anaconda3\envs\civ6_rl_rllib\lib\site-packages\gymnasium\envs\registration.py:498: UserWarning: WARN: Overriding environment FurySimEnv already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
2023-05-18 15:13:49,830	WARNING util.py:67 -- Install gputil for GPU system monitoring.
2023-05-18 15:13:49,837	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory models//test\PPO\PPO_FurySimEnv_9a49c_00000_0_2023-05-18_15-04-24\checkpoint_000012! Trying to extract checkpoint info from other files found in that dir.
(RolloutWorker pid=35304) 2023-05-18 15:13:49,793	WARNING env.py:155 -- Your env doesn't have a .spec.max_episode_steps attribute. Your horizon will default to infinity, and your environment will not be reset.
2023-05-18 15:13:49,849	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_fi

PPO

## Run PPO Model

In [12]:
import numpy as np

for i in range(1):
    observation, info = env.reset()
    terminated = False
    reward = 0
    batch = []
    dps_results = np.array([])
    metrics_batch = []
    debug_logs = []
    debug_logs_batch = []
    state_out = []
    while not terminated:
        action = algorithm.compute_single_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        debug_logs.append(info['debug log'])
    debug_logs_batch.append(debug_logs)
    dps_results = np.append(dps_results, info['dps'])
    metrics_batch.append(info['spell metrics'])

In [16]:
median = np.argsort(dps_results)[len(dps_results)//2]
print(f'Average DPS: {dps_results[median]}')

cast_metrics = metrics_batch[median]
for spell_id, metrics in cast_metrics.items():
    # Only one target, so we can just take the first one
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Average DPS: 8088.5569231609525
Melee: [1, 32, 58]
Death Wish: [2]
Deep Wounds: [111]
Whirlwind: [11]
Recklessness: [1]
Bloodthirst: [15]
Bloodrage: [4]
Whirlwind OH: [11]
Heroic Strike: [26]
Execute: [4]
Slam: [33]
Engi Gloves: [3]


### Debug Log

In [14]:
actions_id = {
    0: "Bloodthirst", 
    1: "Whirlwind",
    2: "Slam",
    3: "HeroicStrike",
    4: "Execute",
    5: "DeathWish",
    6: "Recklessness",
    7: "ShatteringThrow",
    8: "Bloodrage",
    9: "EngiGlove",
    10: "Bloodlust",
    11: "Idle"
}

In [15]:
import pandas as pd

df = pd.DataFrame(debug_logs_batch[median], columns=['Timestamp', 'Action', 'Successful Cast', 'Damage Done', 'Total Damage Done', 'Rage'])
df['Action'] = df['Action'].map(actions_id)
df['DPS'] = df['Total Damage Done'].div(df['Timestamp'])
df

,Timestamp,Action,Successful Cast,Damage Done,Total Damage Done,Rage,DPS
0,0.000000,Bloodthirst,True,7583.504104,7.583504e+03,0.138444,inf
1,0.744206,EngiGlove,True,5825.238291,1.340874e+04,0.746347,1.801751e+04
2,0.744206,EngiGlove,True,0.000000,1.340874e+04,0.746347,1.801751e+04
3,1.500000,Execute,False,0.000000,1.340874e+04,0.746347,8.939162e+03
4,1.744206,Execute,False,659.257222,1.406800e+04,0.746347,8.065559e+03
...,...,...,...,...,...,...,...
477,163.930210,EngiGlove,True,6915.390019,1.298184e+06,0.640626,7.919129e+03
478,164.062964,Recklessness,False,0.000000,1.298184e+06,0.640626,7.912721e+03
479,164.562964,HeroicStrike,True,2365.547630,1.300550e+06,0.640626,7.903054e+03
480,164.979716,Bloodthirst,True,11452.354968,1.312002e+06,0.420626,7.952507e+03


In [16]:
df.to_csv("debug_logs/debug_log_1.csv")

# Spell Debug

In [ ]:
reset()
Spells.register()
Auras.register()
TargetAuras.register()

settings['simOptions']['interactive'] = True
damages = np.array([])
spell_metrics = []

for i in range(1):
    reset()
    while not wowsims.step():
        print(env.reset())
        if wowsims.needsInput():
            wowsims.trySpell(Spells.Bloodthirst)
    totalDamage = wowsims.getDamageDone()
    damages = np.append(damages, totalDamage)
    spell_metrics.append(wowsims.getSpellMetrics())

median = np.argsort(damages)[len(damages)//2]
print(f'Average DPS: {damages[median] / duration}')

In [7]:
settings['simOptions']['interactive'] = True

reset()
while not wowsims.step():
    pass
cast_metrics = spell_metrics[median]
for spell_id, metrics in cast_metrics.items():
    try:
        spell_name = spell_id_lists[spell_id]
    except KeyError as key:
        spell_name = key
    print(f"{spell_name}: {[metric['Casts'] for metric in metrics]}")

Melee: [1, 54]
Deep Wounds: [67]
Heroic Strike: [54]
